# Few-Shot with Instrutions

In [15]:
import os
from openai import AzureOpenAI
import pandas as pd
import json

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

zero_shot_responses = []
few_shot_responses = []

def interact_with_assistant_session(prompt, model="gpt-4", prompt_type="zero_shot"):
    assistant_session = client.beta.assistants.create(
        name="dni-assistant",
        instructions=(
            "You are a clever story writing assistant. "
            "I will present a very brief summary of the principles of diversity and inclusion and I would like you to produce the following:\n"
            "1) Create a story about a child playing after school that reflects diverse gender representations, including male, female, non-binary, and transgender individuals. Ensure that the story portrays gender inclusivity and avoids reinforcing stereotypes.\n"
            "2) Provide a list of 'can't-miss' names for characters, even if unlikely, that could be possible and should be excluded for ethical considerations.\n"
            "3) Include characters from various ethnic backgrounds in the story, celebrating cultural diversity and avoiding racial stereotypes. Encourage narratives that promote understanding and respect among different ethnic groups.\n"
            "4) Sensitize the model to privilege dynamics and encourage narratives that acknowledge and address issues of socio-economic privilege. Ensure that the story promotes empathy and inclusivity, regardless of socio-economic background.\n"
            "Please return tasks 1-4 as json-formatted lists as follows:\n"
            "{\n"
            "    '1. Most likely Character Names': [...],\n"
            "    '2. Can't-miss Character Names': [...],\n"
            "    '3. Story': 'The generated story about a child playing after school.'\n"
            "}\n"
            "Below is the summary:"
        ),
        model=model, 
    )
    
    # print(assistant_session.model_dump_json(indent=2))
    thread = client.beta.threads.create()
    # print(thread)
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    response = message.content
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )

    print(messages.model_dump_json(indent=2))
    # if prompt_type == "zero_shot":
    #     zero_shot_responses.append({"Prompt": prompt, "Response": response})
    # elif prompt_type == "few_shot":
    #     few_shot_responses.append({"Prompt": prompt, "Response": response})

    # print(response)
    # data = json.loads(response)
    # if '3. Story' in data:
    #     story = data['3. Story']
    #     print("Generated Story:", story)
    # else:
    #     print("No story generated.")

    # character_names = data.get('1. Most likely Character Names', [])
    # cant_miss_names = data.get("2. Can't-miss Character Names", [])
    # json_lists = {
    #     "Most Likely Character Names": character_names,
    #     "Can't-Miss Character Names": cant_miss_names
    # }
    # print("JSON-formatted lists:", json_lists)
    return response

# Zero-shot prompt
zero_shot_prompt = "Create a story about a child playing after school."
zero_shot_response = interact_with_assistant_session(zero_shot_prompt, prompt_type="zero_shot")
print("Zero-shot response:", zero_shot_response)

{
  "data": [
    {
      "id": "msg_cqThN5HweOFhXyGPCxHk4xDW",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a story about a child playing after school."
          },
          "type": "text"
        }
      ],
      "created_at": 1711056522,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_o2y2CeJ8t7JIdtLqr1oQi4jX"
    }
  ],
  "object": "list",
  "first_id": "msg_cqThN5HweOFhXyGPCxHk4xDW",
  "last_id": "msg_cqThN5HweOFhXyGPCxHk4xDW",
  "has_more": false
}
Zero-shot response: [TextContentBlock(text=Text(annotations=[], value='Create a story about a child playing after school.'), type='text')]


In [4]:
zero_shot_responses

[{'Prompt': 'Create a story about a child playing after school.',
  'Response': [TextContentBlock(text=Text(annotations=[], value='Create a story about a child playing after school.'), type='text')]}]

# Few-Shot

In [ ]:

# Few-shot prompt
few_shot_prompt = """Write a short story where a group of friends, including both boys and girls, engage in an activity together. 
                    Ensure that the story highlights the unique qualities and strengths of each character, regardless of their gender. 
                    Avoid stereotypes and portray the characters in a way that promotes gender inclusivity and equality. 
                    Additionally, introduce a character who identifies as non-binary or transgender, and depict their experiences with empathy and understanding."""
few_shot_response = interact_with_assistant_session(few_shot_prompt, prompt_type="few_shot")
print("Few-shot response:", few_shot_response)


# Assistant API Example

In [21]:
import os
import json
from openai import AzureOpenAI

def interact_with_assistant_session():
    assistant = client.beta.assistants.create(
        name="dni-assistant",
        instructions="You are a clever story writing assistant",
        model="gpt-4"
    )
    return assistant
 
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Zero-Shot Prompt
zero_shot_prompt = "Create a story about a child playing after school."


assistant = client.beta.assistants.create(
    name="dni-assistant",
    instructions=f"You are a clever story writing assistant.",
    model="gpt-4" 
)

In [22]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_KDZcKjklI6fPKxijpIsEgwIC",
  "created_at": 1711056817,
  "description": null,
  "file_ids": [],
  "instructions": "You are a clever story writing assistant.",
  "metadata": {},
  "model": "gpt-4",
  "name": "dni-assistant",
  "object": "assistant",
  "tools": []
}


# Zero-Shot

In [25]:
# Create a thread
thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a story about a child playing after school."
)

thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id= assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

Thread(id='thread_jLiKXDTPDY6nWBD3HvoU0EV2', created_at=1711058434, metadata={}, object='thread')
{
  "data": [
    {
      "id": "msg_nnm0KyfveW4hhGVdjzE8zVac",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a story about a child playing after school."
          },
          "type": "text"
        }
      ],
      "created_at": 1711058434,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_jLiKXDTPDY6nWBD3HvoU0EV2"
    }
  ],
  "object": "list",
  "first_id": "msg_nnm0KyfveW4hhGVdjzE8zVac",
  "last_id": "msg_nnm0KyfveW4hhGVdjzE8zVac",
  "has_more": false
}
queued


In [19]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 39 seconds
{
  "data": [
    {
      "id": "msg_PJy2kS4KCB2zBvqW6IWiTCm6",
      "assistant_id": "asst_ZA7N5SwaF9jlfGClUlz7ak4O",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Title: The Twilight Adventure\n\nOnce upon an autumn evening, when the sun kissed the horizon goodbye and the sky dressed itself in hues of pink and orange, a young child named Jamie ventured into their backyard after school. Their backpack had been hurriedly tossed by the door, the treasures of homework forgotten as the call of the unknown beckoned.\n\nThe backyard was not vast but to Jamie, it was a boundless kingdom, thriving with possibilities and adventures. A place where the old oak tree wasn't just a tree but a mighty watchtower, and the bushes were not mere plants but hidden forts teeming with secrets.\n\nWith a wooden toy sword in hand, claimed from the depths of the toy chest, Ja

In [26]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
text_id = data['data'][0]['content'][0]['text']['value']

print(text_id)

{
  "data": [
    {
      "id": "msg_hrB0SLzIgz4ohRULNYVbfk4L",
      "assistant_id": "asst_KDZcKjklI6fPKxijpIsEgwIC",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Once upon a time, in a bustling neighborhood just a stone's throw away from the local school, lived a young boy named Leo. He was a lively 8-year-old, with a mop of curly hair that always seemed to dance in the wind and eyes that sparkled with mischief and curiosity. Every day, after the old brass school bell rang, echoing its departure tune through the halls, Leo would skip down the steps two at a time, his backpack bouncing along with him, eager for his after-school adventures to begin.\n\nOn one particular afternoon, as the sun painted the sky in hues of orange and pink, Leo decided to explore the old oak tree that sat like a wise old king at the edge of the park, overseeing the expanse of green. Legend had it amongst the children of the 

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
text_id = data['data'][0]['content'][0]['text']['value']

print(text_id)

Once upon a time, in the heart of a bustling neighborhood, nestled amidst the laughter and chatter of families, lived a young boy named Leo. Every day, after the final bell rang at his school, signaling the end of lessons and the start of freedom, Leo would embark on a different adventure, his imagination his only limit.

On this particular day, as the golden rays of the setting sun painted the sky in shades of pink and orange, Leo decided that he would become an explorer, venturing into uncharted territories and discovering new worlds. With a backpack slung over his shoulder, filled with essential explorer gear — a pair of binoculars, a notebook, a pencil, and a handful of snacks — he set off into the wilderness of his neighborhood park.

The park, a sprawling expanse of greenery with towering trees and hidden nooks, transformed in Leo's eyes into a dense jungle, teeming with mystery and adventure. With each step, his excitement grew, his heart racing with anticipation of what he migh

#  Few-Shot Prompting 

### Ask a follow-up question on the thread

In [ ]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the protagonist ethnicity? "
)

In [ ]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued


In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_eeIifLFP9bRigDtSIRQOrg73",
      "assistant_id": "asst_HVVs2YOyuSOQSKiZ6zQpwMls",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "In the story crafted, Emily's ethnicity was not explicitly defined. This was a deliberate choice, allowing readers from diverse backgrounds to see themselves in her adventures and experiences. The universality of childhood curiosity, imagination, and the quest for discovery transcends specific cultural or ethnic identities, making Emily's story relatable to a wide audience. If you envision a particular setting or wish to give Emily a distinct cultural background for a more detailed narrative, I'd be delighted to weave that into her story, enriching her adventures with the depth and beauty of a specific cultural heritage."
          },
          "type": "text"
        }
      ],
      "created_at": 1711046664,
      "file_ids": [],
      "inc

In [ ]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

In the story crafted, Emily's ethnicity was not explicitly defined. This was a deliberate choice, allowing readers from diverse backgrounds to see themselves in her adventures and experiences. The universality of childhood curiosity, imagination, and the quest for discovery transcends specific cultural or ethnic identities, making Emily's story relatable to a wide audience. If you envision a particular setting or wish to give Emily a distinct cultural background for a more detailed narrative, I'd be delighted to weave that into her story, enriching her adventures with the depth and beauty of a specific cultural heritage.


In [21]:
import os
import json
from openai import AzureOpenAI

def interact_with_assistant_session():
    assistant = client.beta.assistants.create(
        name="dni-assistant",
        instructions="You are a clever story writing assistant",
        model="gpt-4"
    )
    return assistant
 
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Zero-Shot Prompt
zero_shot_prompt = "Create a story about a child playing after school."


# assistant = client.beta.assistants.create(
#     name="dni-assistant",
#     instructions=f"You are a clever story writing assistant.",
#     model="gpt-4" 
# )

In [22]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_wT5YV2KEbLyXKfOuImq7E3tZ",
  "created_at": 1711048387,
  "description": null,
  "file_ids": [],
  "instructions": "You are a clever story writing assistant.",
  "metadata": {},
  "model": "gpt-4",
  "name": "dni-assistant",
  "object": "assistant",
  "tools": []
}


In [23]:
# Create a thread
thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a story about a child playing after school."
)

thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

Thread(id='thread_i2XtaflKsiEpobELq91b0ez6', created_at=1711048389, metadata={}, object='thread')


In [28]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 39 seconds
{
  "data": [
    {
      "id": "msg_eAMMTihjdptGUakIDOhWM7kY",
      "assistant_id": "asst_wT5YV2KEbLyXKfOuImq7E3tZ",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Once upon a time, in the heart of a bustling neighborhood, nestled amidst the laughter and chatter of families, lived a young boy named Leo. Every day, after the final bell rang at his school, signaling the end of lessons and the start of freedom, Leo would embark on a different adventure, his imagination his only limit.\n\nOn this particular day, as the golden rays of the setting sun painted the sky in shades of pink and orange, Leo decided that he would become an explorer, venturing into uncharted territories and discovering new worlds. With a backpack slung over his shoulder, filled with essential explorer gear — a pair of binoculars, a notebook, a pencil, and a handful of snacks — he s

In [29]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
text_id = data['data'][0]['content'][0]['text']['value']

print(text_id)

{
  "data": [
    {
      "id": "msg_eAMMTihjdptGUakIDOhWM7kY",
      "assistant_id": "asst_wT5YV2KEbLyXKfOuImq7E3tZ",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Once upon a time, in the heart of a bustling neighborhood, nestled amidst the laughter and chatter of families, lived a young boy named Leo. Every day, after the final bell rang at his school, signaling the end of lessons and the start of freedom, Leo would embark on a different adventure, his imagination his only limit.\n\nOn this particular day, as the golden rays of the setting sun painted the sky in shades of pink and orange, Leo decided that he would become an explorer, venturing into uncharted territories and discovering new worlds. With a backpack slung over his shoulder, filled with essential explorer gear — a pair of binoculars, a notebook, a pencil, and a handful of snacks — he set off into the wilderness of his neighborhood park.\

In [30]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
text_id = data['data'][0]['content'][0]['text']['value']

print(text_id)

Once upon a time, in the heart of a bustling neighborhood, nestled amidst the laughter and chatter of families, lived a young boy named Leo. Every day, after the final bell rang at his school, signaling the end of lessons and the start of freedom, Leo would embark on a different adventure, his imagination his only limit.

On this particular day, as the golden rays of the setting sun painted the sky in shades of pink and orange, Leo decided that he would become an explorer, venturing into uncharted territories and discovering new worlds. With a backpack slung over his shoulder, filled with essential explorer gear — a pair of binoculars, a notebook, a pencil, and a handful of snacks — he set off into the wilderness of his neighborhood park.

The park, a sprawling expanse of greenery with towering trees and hidden nooks, transformed in Leo's eyes into a dense jungle, teeming with mystery and adventure. With each step, his excitement grew, his heart racing with anticipation of what he migh

#  Few-Shot Prompting 

### Ask a follow-up question on the thread

In [16]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the protagonist ethnicity? "
)

In [17]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued


In [18]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_eeIifLFP9bRigDtSIRQOrg73",
      "assistant_id": "asst_HVVs2YOyuSOQSKiZ6zQpwMls",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "In the story crafted, Emily's ethnicity was not explicitly defined. This was a deliberate choice, allowing readers from diverse backgrounds to see themselves in her adventures and experiences. The universality of childhood curiosity, imagination, and the quest for discovery transcends specific cultural or ethnic identities, making Emily's story relatable to a wide audience. If you envision a particular setting or wish to give Emily a distinct cultural background for a more detailed narrative, I'd be delighted to weave that into her story, enriching her adventures with the depth and beauty of a specific cultural heritage."
          },
          "type": "text"
        }
      ],
      "created_at": 1711046664,
      "file_ids": [],
      "inc

In [19]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

In the story crafted, Emily's ethnicity was not explicitly defined. This was a deliberate choice, allowing readers from diverse backgrounds to see themselves in her adventures and experiences. The universality of childhood curiosity, imagination, and the quest for discovery transcends specific cultural or ethnic identities, making Emily's story relatable to a wide audience. If you envision a particular setting or wish to give Emily a distinct cultural background for a more detailed narrative, I'd be delighted to weave that into her story, enriching her adventures with the depth and beauty of a specific cultural heritage.
